In [1]:
Example 9.1 The following table shows a series of operations and their effects
 on an initially empty priority queue P. The “Priority Queue” column is somewhat
 deceiving since it shows the entries as tuples and sorted by key. Such an internal
 representation is not required of a priority queue.

SyntaxError: invalid character '“' (U+201C) (2147995695.py, line 2)

| Operation        | Return Value | Priority Queue State After Operation |
| ---------------- | ------------ | ------------------------------------ |
| `P.add(5, A)`    | —            | `{(5, A)}`                           |
| `P.add(9, C)`    | —            | `{(5, A), (9, C)}`                   |
| `P.add(3, B)`    | —            | `{(3, B), (5, A), (9, C)}`           |
| `P.add(7, D)`    | —            | `{(3, B), (5, A), (7, D), (9, C)}`   |
| `P.min()`        | `(3, B)`     | `{(3, B), (5, A), (7, D), (9, C)}`   |
| `P.remove_min()` | `(3, B)`     | `{(5, A), (7, D), (9, C)}`           |
| `P.remove_min()` | `(5, A)`     | `{(7, D), (9, C)`                    |
| `len(P)`         | `2`          | `{(7, D), (9, C)}`                   |
| `P.remove_min()` | `(7, D)`     | `{(9, C)}`                           |
| `P.remove_min()` | `(9, C)`     | `{}` (empty)                         |
| `P.is_empty()`   | `True`       | `{}`                                 |
| `P.remove_min()` | `"error"`    | `{}` (still empt\_                   |


Code Fragment 9.1 Description
A PriorityQueueBase class with a nested Item class that composes a key and a value into a single object. For convenience, we provide a concrete implementation of the is_empty() method that is based on a presumed __len__() implementation.

In [ ]:
class PriorityQueueBase:
    """Abstract base class for a priority queue."""

    class Item:
        """Lightweight composite to store priority queue items."""
        __slots__ = '_key', '_value'

        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __lt__(self, other):
            return self._key < other._key  # compare items based on their keys

    def is_empty(self):
        """Return True if the priority queue is empty."""
        return len(self) == 0  # assumes __len__ is implemented in subclass


In [ ]:
k =key and v =value

In [ ]:
Table 9.1 Worst-case running times of the methods of a priority queue of size
 n, realized by means of an unsorted, doubly linked list. The space requirement
 is O(n)

| **Operation**  | **Worst-case Time** | **Explanation**                                                                |
| -------------- | ------------------- | ------------------------------------------------------------------------------ |
| `len()`        | O(1)                | List keeps track of size with a counter.                                       |
| `is_empty()`   | O(1)                | Simply checks if size == 0.                                                    |
| `add(k, v)`    | O(1)                | Inserts at the front or back of the list (no sorting needed).                  |
| `min()`        | O(n)                | Must **scan all items** to find the one with the **smallest key**.             |
| `remove_min()` | O(n)                | Must **scan to find min**, then **remove it**, which takes O(1) after finding. |


In [ ]:
Code Fragment 9.2

In [ ]:

# An implementation of a min-oriented priority queue using an unsorted positional list.
# The parent class PriorityQueueBase is given in Code Fragment 9.1,
# and the PositionalList class is from Section 7.4.

class UnsortedPriorityQueue(PriorityQueueBase):
    """A min-oriented priority queue implemented with an unsorted list."""

    def __init__(self):
        """Create a new empty Priority Queue."""
        self._data = PositionalList()  # internal unsorted positional list

    def __len__(self):
        """Return the number of items in the priority queue."""
        return len(self._data)

    def add(self, key, value):
        """Add a key-value pair."""
        self._data.add_last(self.Item(key, value))  # uses Item from base class

    def _find_min(self):
        """Return Position of item with minimum key (nonpublic utility)."""
        if self.is_empty():
            raise Empty("Priority queue is empty")
        small = self._data.first()
        walk = self._data.after(small)
        while walk is not None:
            if walk.element() < small.element():  # compares keys via Item.__lt__
                small = walk
            walk = self._data.after(walk)
        return small

    def min(self):
        """Return but do not remove (k,v) tuple with minimum key."""
        p = self._find_min()
        item = p.element()
        return (item._key, item._value)

    def remove_min(self):
        """Remove and return (k,v) tuple with minimum key."""
        p = self._find_min()
        item = self._data.delete(p)
        return (item._key, item._value)


In [ ]:
Table 9.2 – Priority Queue Running Times: Unsorted vs. Sorted List

| **Operation**  | **Unsorted List** | **Sorted List** | **Explanation**                                              |
| -------------- | ----------------- | --------------- | ------------------------------------------------------------ |
| `len()`        | O(1)              | O(1)            | List maintains size count.                                   |
| `is_empty()`   | O(1)              | O(1)            | Based on `len() == 0`.                                       |
| `add(k, v)`    | O(1)              | O(n)            | Unsorted: insert at end; Sorted: must find correct position. |
| `min()`        | O(n)              | O(1)            | Unsorted: must scan all; Sorted: min at front.               |
| `remove_min()` | O(n)              | O(1)            | Unsorted: must find min; Sorted: remove front.               |


In [ ]:
Code Fragment 9.3:

In [ ]:

# An implementation of a min-oriented priority queue using a sorted list.
# The parent class PriorityQueueBase is given in Code Fragment 9.1,
# and the PositionalList class is from Section 7.4.

class SortedPriorityQueue(PriorityQueueBase):
    """A min-oriented priority queue implemented with a sorted list."""

    def __init__(self):
        """Create a new empty Priority Queue."""
        self._data = PositionalList()

    def __len__(self):
        """Return the number of items in the priority queue."""
        return len(self._data)

    def add(self, key, value):
        """Add a key-value pair."""
        newest = self.Item(key, value)  # make new item instance
        walk = self._data.last()  # walk backward looking for smaller key
        while walk is not None and newest < walk.element():
            walk = self._data.before(walk)
        if walk is None:
            self._data.add_first(newest)  # new key is smallest
        else:
            self._data.add_after(walk, newest)  # insert after walk

    def min(self):
        """Return but do not remove (k,v) tuple with minimum key."""
        if self.is_empty():
            raise Empty("Priority queue is empty.")
        p = self._data.first()
        item = p.element()
        return (item._key, item._value)

    def remove_min(self):
        """Remove and return (k,v) tuple with minimum key."""
        if self.is_empty():
            raise Empty("Priority queue is empty.")
        item = self._data.delete(self._data.first())
        return (item._key, item._value)


In [ ]:
 Figure 9.1: Example of a heap storing 13 entries with integer keys. The last position
 is the one storing entry (13,W).

In [ ]:
                (4,C)
              /       \
         (5,A)         (15,K)
        /     \        /     \
   (16,X)   (9,F)  (25,J)  (14,E)
   /   \     /  \   /   \
(12,H)(11,S)(6,Z)(7,Q)(13,W)(20,B)


In [ ]:
CodeFragment 9.4: HeapPriorityQueue (Partial) &  Code Fragment 9.5: HeapPriorityQueue (Public Behaviors)

In [ ]:
class HeapPriorityQueue(PriorityQueueBase):
    """A min-oriented priority queue implemented with a binary heap."""

    # ------------------- Nonpublic behaviors -------------------

    def _parent(self, j):
        return (j - 1) // 2

    def _left(self, j):
        return 2 * j + 1

    def _right(self, j):
        return 2 * j + 2

    def _has_left(self, j):
        return self._left(j) < len(self._data)

    def _has_right(self, j):
        return self._right(j) < len(self._data)

    def _swap(self, i, j):
        """Swap the elements at indices i and j of the array."""
        self._data[i], self._data[j] = self._data[j], self._data[i]

    def _upheap(self, j):
        parent = self._parent(j)
        if j > 0 and self._data[j] < self._data[parent]:
            self._swap(j, parent)
            self._upheap(parent)  # recur at parent's position

    def _downheap(self, j):
        if self._has_left(j):
            left = self._left(j)
            small_child = left
            if self._has_right(j):
                right = self._right(j)
                if self._data[right] < self._data[left]:
                    small_child = right
            if self._data[small_child] < self._data[j]:
                self._swap(j, small_child)
                self._downheap(small_child)  # recur at new position
# -------------------- Public behaviors --------------------

def __init__(self):
    """Create a new empty Priority Queue."""
    self._data = []

def __len__(self):
    """Return the number of items in the priority queue."""
    return len(self._data)

def add(self, key, value):
    """Add a key-value pair to the priority queue."""
    self._data.append(self.Item(key, value))
    self._upheap(len(self._data) - 1)  # upheap newly added position

def min(self):
    """Return but do not remove (k,v) tuple with minimum key.
    
    Raise Empty exception if empty.
    """
    if self.is_empty():
        raise Empty("Priority queue is empty.")
    item = self._data[0]
    return (item._key, item._value)

def remove_min(self):
    """Remove and return (k,v) tuple with minimum key.
    
    Raise Empty exception if empty.
    """
    if self.is_empty():
        raise Empty("Priority queue is empty.")
    self._swap(0, len(self._data) - 1)   # put min item at the end
    item = self._data.pop()              # remove it from list
    self._downheap(0)                    # fix new root
    return (item._key, item._value)


In [3]:
 Table 9.3: Performance of a Heap-Based Priority Queue

SyntaxError: invalid syntax (3270772704.py, line 1)

| Operation        | Running Time |
| ---------------- | ------------ |
| `len(P)`         | O(1)         |
| `P.is_empty()`   | O(1)         |
| `P.min()`        | O(1)         |
| `P.add()`        | O(log n)\*   |
| `P.remove_min()` | O(log n)\*   |


In [ ]:
 Table 9.3: Performance of a priority queue, P, realized by means of a heap. We
 let n denote the number of entries in the priority queue at the time an operation is
 executed. The space requirement is O(n). The running time of operations min and
 remove min are amortized for an array-based representation, due to occasional re
sizing of a dynamic array; those bounds are worst case with a linked tree structure.

In [ ]:
 Code Fragment 9.6: HeapPriorityQueue with Bulk Initialization

In [ ]:
def __init__(self, contents=()):
    """Create a new priority queue.
    
    By default, the queue will be empty. If `contents` is given,
    it should be an iterable sequence of (k, v) tuples specifying the initial contents.
    """
    self._data = [self.Item(k, v) for k, v in contents]
    if len(self._data) > 1:
        self._heapify()

def _heapify(self):
    """Perform bottom-up heap construction in linear time."""
    start = self._parent(len(self._data) - 1)  # start at parent of last leaf
    for j in range(start, -1, -1):             # down to and including the root
        self._downheap(j)


In [6]:
 Python heapq Module: Key Functions and Their Running Times

SyntaxError: invalid syntax (612918557.py, line 1)

| Function                 | Description                                                                                                             | Time Complexity    |
| ------------------------ | ----------------------------------------------------------------------------------------------------------------------- | ------------------ |
| `heappush(L, e)`         | Push element `e` onto list `L` and restore the heap-order property.                                                     | **O(log n)**       |
| `heappop(L)`             | Pop and return the smallest element from heap `L`.                                                                      | **O(log n)**       |
| `heappushpop(L, e)`      | Push `e` onto `L`, then pop and return the smallest item. More efficient than calling `heappush` followed by `heappop`. | **O(log n)**       |
| `heapreplace(L, e)`      | Pop the smallest item from `L`, then push `e`. Ensures the new element is **not** returned even if it's the smallest.   | **O(log n)**       |
| `heapify(L)`             | Transform an unordered list `L` into a valid heap (min-heap). Uses a bottom-up approach.                                | **O(n)**           |
| `nlargest(k, iterable)`  | Return a list of the `k` largest elements from `iterable`.                                                              | **O(n + k log n)** |
| `nsmallest(k, iterable)` | Return a list of the `k` smallest elements from `iterable`.                                                             | **O(n + k log n)** |


In [ ]:
Code Fragment 9.7: pq_sort using a Priority Queue

In [ ]:
def pq_sort(C):
    """Sort a collection of elements stored in a positional list using a priority queue."""
    n = len(C)
    P = PriorityQueue()  # assume a min-oriented priority queue

    # Phase 1: Insert each element into the priority queue
    for j in range(n):
        element = C.delete(C.first())
        P.add(element, element)  # use element as both key and value

    # Phase 2: Extract elements in sorted order back into C
    for j in range(n):
        (k, v) = P.remove_min()
        C.add_last(v)


In [ ]:
Selection Sort Using a Priority Queue
📥 Input:
Initial collection C: (7, 4, 8, 2, 5, 3)

Priority queue P: ()

In [ ]:
 Phase 1: Move All Elements to Priority Queue

| Step | Collection C    | Priority Queue P   |
| ---- | --------------- | ------------------ |
| (a)  | (4, 8, 2, 5, 3) | (7)                |
| (b)  | (8, 2, 5, 3)    | (7, 4)             |
| (c)  | (2, 5, 3)       | (7, 4, 8)          |
| (d)  | (5, 3)          | (7, 4, 8, 2)       |
| (e)  | (3)             | (7, 4, 8, 2, 5)    |
| (f)  | ()              | (7, 4, 8, 2, 5, 3) |


In [ ]:
Phase 2: Remove Minimums Back into Collection (Sorted)

| Step | Collection C       | Priority Queue P |
| ---- | ------------------ | ---------------- |
| (a)  | (2)                | (7, 4, 8, 5, 3)  |
| (b)  | (2, 3)             | (7, 4, 8, 5)     |
| (c)  | (2, 3, 4)          | (7, 8, 5)        |
| (d)  | (2, 3, 4, 5)       | (7, 8)           |
| (e)  | (2, 3, 4, 5, 7)    | (8)              |
| (f)  | (2, 3, 4, 5, 7, 8) | ()               |


In [ ]:
 Insertion Sort via Priority Queue
📥 Input:
Initial Collection C: (7, 4, 8, 2, 5, 3)

Initial Priority Queue P: ()

In [ ]:
Phase 1: Insert All Elements into Priority Queue

| Step | Collection C    | Priority Queue P   |
| ---- | --------------- | ------------------ |
| (a)  | (4, 8, 2, 5, 3) | (7)                |
| (b)  | (8, 2, 5, 3)    | (4, 7)             |
| (c)  | (2, 5, 3)       | (4, 7, 8)          |
| (d)  | (5, 3)          | (2, 4, 7, 8)       |
| (e)  | (3)             | (2, 4, 5, 7, 8)    |
| (f)  | ()              | (2, 3, 4, 5, 7, 8) |


In [ ]:
Phase 2: Remove Minimums Back into Collection (Sorted)

| Step | Collection C       | Priority Queue P |
| ---- | ------------------ | ---------------- |
| (a)  | (2)                | (3, 4, 5, 7, 8)  |
| (b)  | (2, 3)             | (4, 5, 7, 8)     |
| (c)  | (2, 3, 4)          | (5, 7, 8)        |
| (d)  | (2, 3, 4, 5)       | (7, 8)           |
| (e)  | (2, 3, 4, 5, 7)    | (8)              |
| (f)  | (2, 3, 4, 5, 7, 8) | ()               |


In [ ]:
 P.update(loc, k, v)
Purpose: Updates the key and value of the item at the given locator loc.

Parameters:

loc: The locator identifying the item in the priority queue.

k: The new key.

v: The new value.

Effect: Replaces the current key-value pair at loc with the new (k, v), and restores the heap-order property if violated.

Use Case: When the priority of a task changes and needs to be repositioned in the queue.

In [ ]:
P.remove(loc)
Purpose: Removes the item identified by the locator loc from the priority queue.

Parameters:

loc: The locator pointing to the item to remove.

Returns: The (key, value) pair of the removed item.

Effect: Deletes the entry from the structure and reorders the heap to maintain validity.

Use Case: Cancelling a task or removing a job that’s no longer needed.

In [13]:
Code Fragment 9.8: AdaptableHeapPriorityQueue & 9.9 an implemention of an adapatable priority queue(Part 1)
An implementation of an adaptable priority queue using a binary heap, which extends the HeapPriorityQueue class from Code Fragments 9.4 and 9.5. It introduces a Locator class to enable efficient updates and removals by maintaining the position (index) of each entry.

python
Copy
Edit
class AdaptableHeapPriorityQueue(HeapPriorityQueue):
    """A locator-based priority queue implemented with a binary heap."""

    # ------------------------------ nested Locator class ------------------------------
    class Locator(HeapPriorityQueue.Item):
        """Token for locating an entry of the priority queue."""
        __slots__ = '_index'  # add index as additional field

        def __init__(self, k, v, j):
            super().__init__(k, v)
            self._index = j

    # ------------------------------ nonpublic behaviors ------------------------------
    # override swap to record new indices
    def swap(self, i, j):
        super().swap(i, j)                    # perform the swap
        self._data[i]._index = i              # reset locator index (post-swap)
        self._data[j]._index = j

    def bubble(self, j):
        if j > 0 and self._data[j] < self._data[self.parent(j)]:
            self.upheap(j)
        else:
            self.downheap(j)

class AdaptableHeapPriorityQueue(HeapPriorityQueue):
    """A locator-based priority queue implemented with a binary heap."""

    # ------------------------------ nested Locator class ------------------------------
    class Locator(HeapPriorityQueue.Item):
        """Token for locating an entry of the priority queue."""
        __slots__ = '_index'

        def __init__(self, k, v, j):
            super().__init__(k, v)
            self._index = j

    # ------------------------------ nonpublic behaviors ------------------------------
    def swap(self, i, j):
        super().swap(i, j)
        self._data[i]._index = i
        self._data[j]._index = j

    def bubble(self, j):
        if j > 0 and self._data[j] < self._data[self.parent(j)]:
            self.upheap(j)
        else:
            self.downheap(j)

    # ------------------------------ public behaviors ------------------------------
    def add(self, key, value):
        """Add a key-value pair and return a locator."""
        token = self.Locator(key, value, len(self._data))
        self._data.append(token)
        self.upheap(len(self._data) - 1)
        return token

    def update(self, loc, newkey, newval):
        """Update the key and value for the entry identified by Locator loc."""
        j = loc._index
        if not (0 <= j < len(self) and self._data[j] is loc):
            raise ValueError("Invalid locator")
        loc._key = newkey
        loc._value = newval
        self.bubble(j)

    def remove(self, loc):
        """Remove and return the (k,v) pair identified by Locator loc."""
        j = loc._index
        if not (0 <= j < len(self) and self._data[j] is loc):
            raise ValueError("Invalid locator")
        if j == len(self) - 1:
            self._data.pop()
        else:
            self.swap(j, len(self._data) - 1)
            self._data.pop()
            self.bubble(j)
        return (loc._key, loc._value)


SyntaxError: invalid syntax (2995672376.py, line 1)

**Table 9.4**: Running times of the methods of an adaptable priority queue, P, of size *n*,  
realized by means of an array-based heap representation. The space requirement is *O(n)*.

| Operation                     | Running Time |
|------------------------------|---------------|
| `len(P)`, `P.is_empty()`, `P.min()` | O(1)         |
| `P.add(k, v)`                | O(log n)\*    |
| `P.update(loc, k, v)`        | O(log n)      |
| `P.remove(loc)`              | O(log n)\*    |
| `P.remove_min()`             | O(log n)\*    |

\*Amortized with dynamic array


In [ ]:
# R-9.1
"""
How long would it take to remove the logn smallest elements from a
heap that contains n entries, using the remove_min operation?
"""


In [ ]:
Θ((logn)^2)

In [ ]:
# R-9.2
"""
Suppose you label each position p of a binary tree T with a key equal to
its preorder rank. Under what circumstances is T a heap?
"""


In [ ]:
The heap-order property holds automatically (preorder visits a parent before every node in its subtree, so parent key < child keys). Therefore, T is a min-heap iff its shape is a complete binary tree. In other words, with preorder ranks as keys, the only additional requirement for being a heap is completeness.

In [ ]:
# R-9.3
"""
What does each remove_min call return within the following sequence of
priority queue ADT methods:
add(5,A), add(4,B), add(7,F), add(1,D),
remove_min(), add(3,J), add(6,L), remove_min(),
remove_min(), add(8,G), remove_min(), add(2,H),
remove_min(), remove_min()?
"""


In [ ]:
(1, D)

(3, J)

(4, B)

(5, A)

(2, H)

(6, L)

In [ ]:
# R-9.4
"""
An airport is developing a computer simulation of air-traffic control that
handles events such as landings and takeoffs. Each event has a time stamp
that denotes the time when the event will occur.

The simulation program needs to efficiently perform the following two operations:
• Insert an event with a given time stamp (add a future event).
• Extract the event with smallest time stamp (next event to process).

Which data structure should be used for the above operations? Why?
"""


In [ ]:
min-heap priority queue

In [ ]:
# R-9.5
"""
The min method for the UnsortedPriorityQueue class executes in O(n)
time, as analyzed in Table 9.2. Give a simple modification to the class so
that min runs in O(1) time. Explain any necessary modifications to other
methods of the class.
"""


In [ ]:
To make min() run in O(1) for an UnsortedPriorityQueue, you can store a reference to the current minimum element when items are added or removed.

In [ ]:
# R-9.6
"""
Can you adapt your solution to the previous problem to make remove_min
run in O(1) time for the UnsortedPriorityQueue class? Explain your answer.
"""


In [ ]:
Yes — by maintaining a direct reference to the minimum element and storing elements in a data structure that allows O(1) deletion (such as a linked list with a pointer to the minimum), remove_min() could run in O(1), but this requires updating the minimum reference on every add() and carefully adjusting it on removals, which can make add() potentially O(n) if the removed minimum needs to be replaced by scanning the remaining elements.

In [ ]:
# R-9.7
"""
Illustrate the execution of the selection-sort algorithm on the following
input sequence:
(22, 15, 36, 44, 10, 3, 9, 13, 29, 25)
"""


In [26]:
# Selection sort step-by-step illustration for the given sequence

arr = [22, 15, 36, 44, 10, 3, 9, 13, 29, 25]
steps = []

# Perform selection sort and record steps
for i in range(len(arr)):
    min_index = i
    for j in range(i + 1, len(arr)):
        if arr[j] < arr[min_index]:
            min_index = j
    arr[i], arr[min_index] = arr[min_index], arr[i]
    steps.append(arr.copy())

    print(f"Pass {i + 1}: {arr}")
    



Pass 1: [3, 15, 36, 44, 10, 22, 9, 13, 29, 25]
Pass 2: [3, 9, 36, 44, 10, 22, 15, 13, 29, 25]
Pass 3: [3, 9, 10, 44, 36, 22, 15, 13, 29, 25]
Pass 4: [3, 9, 10, 13, 36, 22, 15, 44, 29, 25]
Pass 5: [3, 9, 10, 13, 15, 22, 36, 44, 29, 25]
Pass 6: [3, 9, 10, 13, 15, 22, 36, 44, 29, 25]
Pass 7: [3, 9, 10, 13, 15, 22, 25, 44, 29, 36]
Pass 8: [3, 9, 10, 13, 15, 22, 25, 29, 44, 36]
Pass 9: [3, 9, 10, 13, 15, 22, 25, 29, 36, 44]
Pass 10: [3, 9, 10, 13, 15, 22, 25, 29, 36, 44]


In [ ]:
# R-9.8
"""
Illustrate the execution of the insertion-sort algorithm on the input sequence
from the previous problem.
"""


In [28]:
def insertion_sort_verbose(arr):
    arr = arr.copy()  # avoid modifying original
    n = len(arr)
    
    for i in range(1, n):
        current = arr[i]
        j = i - 1
        while j >= 0 and arr[j] > current:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = current
        
        print(f"Pass {i}: {arr}")

# Example input
data = [22, 15, 36, 44, 10, 3, 9, 13, 29, 25]
insertion_sort_verbose(data)


Pass 1: [15, 22, 36, 44, 10, 3, 9, 13, 29, 25]
Pass 2: [15, 22, 36, 44, 10, 3, 9, 13, 29, 25]
Pass 3: [15, 22, 36, 44, 10, 3, 9, 13, 29, 25]
Pass 4: [10, 15, 22, 36, 44, 3, 9, 13, 29, 25]
Pass 5: [3, 10, 15, 22, 36, 44, 9, 13, 29, 25]
Pass 6: [3, 9, 10, 15, 22, 36, 44, 13, 29, 25]
Pass 7: [3, 9, 10, 13, 15, 22, 36, 44, 29, 25]
Pass 8: [3, 9, 10, 13, 15, 22, 29, 36, 44, 25]
Pass 9: [3, 9, 10, 13, 15, 22, 25, 29, 36, 44]


In [ ]:
# R-9.9
"""
Give an example of a worst-case sequence with n elements for insertion sort,
and show that insertion sort runs in Ω(n^2) time on such a sequence.
"""


In [ ]:
worst case being a sequence for insertion sort, where the it is a stritly decreasing list
[n,n-1,n-2,.......2,1]

In [ ]:
# R-9.10
"""
At which positions of a heap might the third smallest key be stored?
"""


In [ ]:
At Level 1 and Level 2, it cannot be deeper than level 2

In [ ]:
# R-9.11
"""
At which positions of a heap might the largest key be stored?
"""


In [ ]:
the largest will be located at one of the leaf positions, since the heap property ensures that every parent's key is less than or equal to its childrens key
the largest key must be at an index of n/2 +1 to n 
for an example n=15, the largest key must be at an index of 8-15

In [ ]:
# R-9.12
"""
Consider a situation in which a user has numeric keys and wishes to have
a priority queue that is maximum-oriented. How could a standard (min-oriented)
priority queue be used for such a purpose?
"""


In [ ]:
Use a regular min-heap but store each key as its negative: insert (-k, value) instead of (k, value).
Then the min-heap’s remove_min() returns the pair with the largest original key (recover k = -stored_key).
Equivalently, you can wrap entries with a comparator that reverses the order (i.e., compare by -key).

In [ ]:
# R-9.13
"""
Illustrate the execution of the in-place heap-sort algorithm on the following
input sequence:
(2, 5, 16, 4, 10, 23, 39, 18, 26, 15)
"""


In [33]:
import heapq

def heap_sort_in_place_trace(arr):
    a = arr.copy()
    
    # Step 1: Build a max heap (Python's heapq is min-heap, so we store negatives)
    max_heap = [-x for x in a]
    heapq.heapify(max_heap)
    print("Step 1 - Build max heap:", [-x for x in max_heap])
    
    # Step 2: Perform heap sort
    sorted_array = []
    for i in range(len(a)):
        max_val = -heapq.heappop(max_heap)
        sorted_array.insert(0, max_val)  # simulate placing max at end
        print(f"Step {i+2} - Extract max {max_val}:", sorted_array)

# Given sequence
sequence = [2, 5, 16, 4, 10, 23, 39, 18, 26, 15]
heap_sort_in_place_trace(sequence)


Step 1 - Build max heap: [39, 26, 23, 18, 15, 2, 16, 5, 4, 10]
Step 2 - Extract max 39: [39]
Step 3 - Extract max 26: [26, 39]
Step 4 - Extract max 23: [23, 26, 39]
Step 5 - Extract max 18: [18, 23, 26, 39]
Step 6 - Extract max 16: [16, 18, 23, 26, 39]
Step 7 - Extract max 15: [15, 16, 18, 23, 26, 39]
Step 8 - Extract max 10: [10, 15, 16, 18, 23, 26, 39]
Step 9 - Extract max 5: [5, 10, 15, 16, 18, 23, 26, 39]
Step 10 - Extract max 4: [4, 5, 10, 15, 16, 18, 23, 26, 39]
Step 11 - Extract max 2: [2, 4, 5, 10, 15, 16, 18, 23, 26, 39]


In [ ]:
# R-9.14
"""
Let T be a complete binary tree such that position p stores an element
with key f(p), where f(p) is the level number of p.
Is tree T a heap? Why or why not?
"""


In [ ]:
Yes—T is a min-heap.Since each node’s key is its level number, a parent at level $\ell$ has key $\ell$ and each child at level $\ell+1$ has key $\ell+1$. Thus every parent key is strictly less than its children’s keys (heap-order property). Given T is also complete (heap shape property), T satisfies all conditions for a min-heap.


In [ ]:
# R-9.15
"""
Explain why the description of down-heap bubbling does not consider the
case in which position p has a right child but not a left child.
"""


In [ ]:
Because in a complete binary tree (which is the underlying structure of a heap), nodes are filled level by level from left to right, it’s impossible for a node to have a right child without having a left child first.
So when down-heap bubbling is described, the possible child configurations are:

No children (stop bubbling).

Only a left child.

Both left and right children.

The “right child only” case can’t occur in a proper heap structure.

In [ ]:
# R-9.16
"""
Is there a heap H storing seven entries with distinct keys such that a preorder
traversal of H yields the entries of H in increasing or decreasing
order by key? How about an inorder traversal? How about a postorder traversal?
If so, give an example; if not, say why.
"""


In [ ]:
Preorder:
The preorder traversal can be increasing in a min-heap. For example, consider a min-heap with level-order array [1, 2, 5, 3, 4, 6, 7]. The preorder of this tree is 1, 2, 3, 4, 5, 6, 7 (increasing), and it satisfies the heap order since each parent is less than its children.
The preorder traversal cannot be decreasing in a min-heap, because preorder visits the root first, and the root is the smallest key, not the largest.

Inorder:
The inorder traversal cannot be strictly increasing or decreasing in a min-heap. Inorder’s middle visit is the root, but in a min-heap the root must be the minimum, which cannot be the middle element of a strictly monotone inorder sequence when there are two or more levels.

Postorder:
The postorder traversal can be decreasing in a min-heap. For example, consider a min-heap with level-order array [1, 5, 2, 7, 6, 4, 3]. The postorder of this tree is 7, 6, 5, 4, 3, 2, 1 (decreasing), and it satisfies the heap order.
The postorder traversal cannot be increasing in a min-heap, because postorder visits the root last, and the root is the smallest key, not the largest.

In [ ]:
# R-9.17
"""
Let H be a heap storing 15 entries using the array-based representation of
a complete binary tree. What is the sequence of indices of the array that
are visited in a preorder traversal of H? What about an inorder traversal?
What about a postorder traversal?
"""


In [ ]:
Assuming the usual 1-based heap indexing (root at 1; children of i at 2i and 2i+1) for a complete binary tree with 15 nodes:

Preorder (root, left, right):
1, 2, 4, 8, 9, 5, 10, 11, 3, 6, 12, 13, 7, 14, 15

Inorder (left, root, right):
8, 4, 9, 2, 10, 5, 11, 1, 12, 6, 13, 3, 14, 7, 15

Postorder (left, right, root):
8, 9, 4, 10, 11, 5, 2, 12, 13, 6, 14, 15, 7, 3, 1

In [ ]:
# R-9.19
"""
Bill claims that a preorder traversal of a heap will list its keys in nondecreasing order.
Draw an example of a heap that proves him wrong.
"""


In [ ]:
       1
     /   \
    4     3
   / \   / \
  7   5 6   8
1,4,7,5,3,6,8 This sequence is not nondecreasing because 4>3 appears partway through.

In [ ]:
# R-9.20
"""
Hillary claims that a postorder traversal of a heap will list its keys in nonincreasing order.
Draw an example of a heap that proves her wrong.
"""


In [ ]:
         1
       /   \
      3     2
     / \   / \
    7   6 5   4
This proves Hillary wrong because postorder traversal does not necessarily list heap elements in nonincreasing order, even though the heap satisfies all heap properties.

In [ ]:
# R-9.21
"""
Show all the steps of the algorithm for removing the entry (16, X) from the
heap of Figure 9.1, assuming the entry had been identified with a locator.
"""


In [ ]:
1. Locate the node

The locator identifies (16, X) directly, which is a leaf node in the leftmost position under (15, K).

2. Swap with last item

The last entry in the heap (based on the array representation) is (20, B), found as the right child of (6, Z).

Swap (16, X) with (20, B).

3. Remove the last item

Remove (16, X) from the last position. This completes the deletion step.

4. Restore heap order

Now check the new position of (20, B) — it is now a left child of (15, K).

Compare with its parent:

(20, B) > (15, K) → Heap property is preserved (in min-heap: parent < children).

Only the value (16, X) is gone; (20, B) took its place. All heap order properties are maintained because the value 20 is still larger than its parent 15, and has no children of its own.

In [ ]:
# R-9.22
"""
Show all the steps of the algorithm for replacing the key of entry (5, A) with
18 in the heap of Figure 9.1, assuming the entry had been identified with
a locator.
"""


In [2]:
Step 1: Identify the entry (5, A)

The locator points directly to node (5, A). It is a left child of (4, C).

Step 2.

We change the key 5 to 18. So the entry becomes (18, A).

At this point, the heap may no longer satisfy the min-heap property, because 18 > 15, 18 > 9, etc.

We must restore the heap property by pushing the newly updated entry (18, A) down the tree if its key is greater than its children.

Level 1:

Parent: (18, A)

Children: (15, K) and (9, F)

Smallest child: (9, F)

Level 2:

Now (18, A) is at the position of (9, F)

Children: (14, E) and (12, H)

Smallest child: (12, H)

Level 3:

Now (18, A) is at a leaf. No more children

After the operation, entries move as follows:

(5, A) → (18, A)

Swaps:

(18, A) ↔ (9, F)

(18, A) ↔ (12, H)

Final location of (18, A): same place where (12, H) used to be.


SyntaxError: invalid syntax (2219867057.py, line 1)

In [ ]:
# R-9.23
"""
Draw an example of a heap whose keys are all the odd numbers from 1 to 59
(no repeats), such that the insertion of an entry with key 32 would cause
up-heap bubbling to proceed all the way up to a child of the root
(replacing that child’s key with 32).
"""


In [4]:
Before:
[1, 40, 3, 41, 43, 5, 7, 45, 47, 49, 51, 9, 11, 13, 15, 53, 55, 57, 59, 17, 19, 21, 23, 25, 27, 29, 31]

After:

[1, 40, 3, 50, 43, 5, 7, 55, 47, 49, 51, 9, 11, 13, 15, 57, 59, 45, 17, 19, 21, 23, 25, 27, 29, 31, 33]


SyntaxError: invalid syntax (1757771946.py, line 1)

In [ ]:
# R-9.24
"""
Describe a sequence of n insertions in a heap that requires Ω(n log n) time
to process.
"""


In [ ]:
Inserting a reverse-sorted sequence (e.g., 1000, 999, 998, ..., 2, 1) into a min-heap triggers up-heap bubbling for every insertion, resulting in Ω(n log n) total time.

In [ ]:
# R-9.25
"""
Complete Figure 9.9 by showing all the steps of the in-place heap-sort algorithm.
Show both the array and the associated heap at the end of each step.
"""


In [ ]:
Original: [9, 7, 5, 2, 6, 4]

Heap tree:
        9
      /   \
     7     5
    / \   /
   2   6 4

[4, 7, 5, 2, 6, 9] — Max element 9 sorted
Down-heap 4 → 7 → 6 → final position at index 1

        7
      /   \
     6     5
    / \
   2   4
[4, 6, 5, 2, 7, 9] — 7 sorted
Down-heap 4 → 6 → final position at index 1

        6
      /   \
     4     5
    / 
   2

[2, 4, 5, 6, 7, 9] — 6 sorted
Down-heap 2 → 5 → final position at index 2

        5
      /   \
     4     2
[2, 4, 5, 6, 7, 9] — 5 sorted
Down-heap 2 → 4 → final position at index 1
        4
      /
     2
[2, 4, 5, 6, 7, 9] — 4 sorted
Remaining element 2 is already in place.

[2, 4, 5, 6, 7, 9]


In [ ]:
Table 9.2 – Priority Queue Running Times: Unsorted vs. Sorted List

| **Operation**  | **Unsorted List** | **Sorted List** | **Explanation**                                              |
| -------------- | ----------------- | --------------- | ------------------------------------------------------------ |
| `len()`        | O(1)              | O(1)            | List maintains size count.                                   |
| `is_empty()`   | O(1)              | O(1)            | Based on `len() == 0`.                                       |
| `add(k, v)`    | O(1)              | O(n)            | Unsorted: insert at end; Sorted: must find correct position. |
| `min()`        | O(n)              | O(1)            | Unsorted: must scan all; Sorted: min at front.               |
| `remove_min()` | O(n)              | O(1)            | Unsorted: must find min; Sorted: remove front.               |
